# 环境配置

## 登录

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/YOUR_PATH"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['data', 'Output', 'run', '8b.ipynb', 'api_used.ipynb']

In [ ]:
from huggingface_hub import login
login()

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## 安装

In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install nltk

In [ ]:
pip install openai

## 初始化

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "YOUR_API"
)

In [ ]:
import argparse
import json
import re
import os
import time
import numpy as np
from datetime import datetime
from typing import List
from typing import Tuple
import torch
import csv
import random
import nltk
nltk.download('punkt')
from rich import print
import transformers
#from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import random
# 种群
from data.mutation_prompts import mutation_prompts
from data.thinking_styles import thinking_styles
from data.types import EvolutionUnit, Population
from data import gsm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


create population

# 函数

In [ ]:
# 数据集
gsm8k_examples = gsm.read_jsonl('data/dataset/SVAMP_test.jsonl')

# population设定
simulations = 3
problem_description = 'Solve the math word problem, giving your answer as an arabic numeral.'
num_mutation_prompts = 2
num_thinking_styles = 4
num_evals = 1

tp_set = mutation_prompts[:num_mutation_prompts]
mutator_set = thinking_styles[:num_thinking_styles]
question_set = gsm8k_examples[6:11]


In [ ]:
data = {
        'size': len(tp_set)*len(mutator_set),
        'age': 0,
        'problem_description': problem_description,
        'elites': [],
        'units': [EvolutionUnit(**{
            'T': t,
            'M': m,
            'P': '',
            'fitness': 0,
            'history': []
        }) for t in tp_set for m in mutator_set],
        'result_dir': ''
    }
population=Population(**data)

In [ ]:
#print(question_set)

In [ ]:
prompts = []
result_list = []

## result processing

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda:0",
)
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def handle_result(system_prompt=prompts, batch_example=question_set):
  result_list = []
  part_prompt="The answer should be an equation contains an unknown variable x, a solution number with one decimal place that satisfies the equation, connected with '=>'. eg. '70-x=27 => 43.0'"
  for count, prompt in enumerate(system_prompt):
    for index, example in enumerate(batch_example):
      description = example['question']
      outputs=[]
      messages = [
          {"role": "system", "content": str(prompt)+part_prompt},
          {"role": "user", "content":description},]
      outputs=pipeline(
          messages,
          max_new_tokens=256,
          eos_token_id=terminators,
          do_sample=True,
          temperature=0.6,
          top_p=0.9,)
      print(outputs[0]["generated_text"][-1]['content'])
      result_list.append(outputs[0]["generated_text"][-1]['content'])
  return result_list

## evaluate

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
def evaluate_fitness(population=population, results=result_list, batch=question_set,num_evals=num_evals):
  start_time = time.time()
  now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  filename = f"{now}.csv"
  result_dir = population.result_dir
  filepath = os.path.join(result_dir, filename)

  with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['unit_index', 'question', 'generated_answer',
            'correct_answer', 'prompt', 'total_fitness',
            'similarity', 'complexity','log_perplexity']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # 初始化fitness
    for unit in population.units:
      unit.fitness = 0
    # 开始评分
    for unit_index, fitness_results in enumerate(results):
      #for i, x in enumerate(fitness_results):
      #generated_answer = fitness_results['content']
      generated_answer = fitness_results
      print(f"正在评价结果：{unit_index}")
      correct_answer = batch[i%num_evals]['answer']
      question = batch[i%num_evals]['question']

      # similarity evaluation
      model.encode(generated_answer)
      similarity = 1 - cosine(model.encode(generated_answer), model.encode(correct_answer))

      # Complexity evaluation
      words = generated_answer.split()
      length = len(words)
      sentences = nltk.sent_tokenize(generated_answer)
      grammar_complexity = sum(len(nltk.word_tokenize(sentence)) for sentence in sentences)
      logic_steps = generated_answer.count('.')

      # Perplexity evaluation
      gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
      gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2-large')
      tokenize_input = gpt2_tokenizer.encode(generated_answer, return_tensors="pt")
      loss = gpt2_model(tokenize_input, labels=tokenize_input).loss
      perplexity=torch.exp(loss).item()
      log_perplexity = np.log(perplexity + 0.1)

      # 综合评价
      weights = {
        'similarity_weight': 1,
        'complexity_weight': 0,
        'log_perplexity_weight': 0
      }
      unit.fitness += (
        weights['similarity_weight'] * similarity +
        weights['complexity_weight'] * (length + grammar_complexity + logic_steps)
        + weights['log_perplexity_weight'] * log_perplexity
      )

      # 记录日志和结果
      writer.writerow({
        'unit_index': unit_index,
        'question': question,
        'generated_answer': generated_answer,
        'correct_answer': correct_answer,
        'prompt': unit.P,
        'total_fitness': unit.fitness,
        'similarity': similarity,
        'log_perplexity': log_perplexity,
        'complexity': length + grammar_complexity + logic_steps
      })
      # 记录种群的最佳效果
      elite_fitness = max(unit.fitness for unit in population.units)
      population.elites.extend([unit for unit in population.units if unit.fitness == elite_fitness])

  end_time = time.time()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## mutate

In [ ]:
def zero_order_prompt_gen(unit: EvolutionUnit, problem_description: str, **kwargs) -> EvolutionUnit:
    completion = client.chat.completions.create(
    model="meta/llama3-8b-instruct",
    messages=[
        {"role":"user",
        "content":"Please generate a short prompt based on the following(Less than 25 tokens)."+problem_description + " An ordered list of 50 hints: "}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )
    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    result=''.join(mylist)

    # search for the pattern "anything after 1. and before 2."
    pattern = r"1\.(.*?)2\."
    match = re.search(pattern, result, re.DOTALL)
    if match:
        # return the first match
        unit.P = match.group(1).strip()
    else:
        unit.P = ""

    return unit


def first_order_prompt_gen(unit: EvolutionUnit, **kwargs) -> EvolutionUnit:

    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[{"role":"user","content":"Please generate a short prompt based on the following(Less than 25 tokens)."+str(unit.M) + " " + str(unit.P)}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )
    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp=''.join(mylist)
    unit.P=temp
    return unit


def lineage_based_mutation(unit: EvolutionUnit, elites: List[EvolutionUnit], **kwargs) -> EvolutionUnit:
    HEADING = "GENOTYPES FOUND IN ASCENDING ORDER OF QUALITY \n "
    # made a choice not to format it with newlines, could change later.
    ITEMS = "\n".join(["{}. {}".format(i + 1, x.P) for i, x in enumerate(elites)])

    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[
        {"role":"user","content":"Please generate a short prompt based on the following(Less than 25 tokens)."+HEADING + ITEMS}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )
    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp=''.join(mylist)
    unit.P=temp

    return unit


# Hypermutation
def zero_order_hypermutation(unit: EvolutionUnit, problem_description: str, **kwargs) -> EvolutionUnit:
    RANDOM_THINKING_STYLE = random.sample(thinking_styles, 1)[0]
    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[{"role":"user",
          "content":"Please generate a short prompt based on the following(Less than 25 tokens)."+problem_description + " " + RANDOM_THINKING_STYLE}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )

    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp=''.join(mylist)
    unit.M=temp
    return unit


def first_order_hypermutation(unit: EvolutionUnit, **kwargs) -> EvolutionUnit:
    HYPER_MUTATION_PROMPT = "Please summarize and improve the following instruction: "
    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[{"role":"user",
          "content":"Please generate a short prompt based on the following(Less than 25 tokens)."+HYPER_MUTATION_PROMPT + str(unit.M)}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )

    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp=''.join(mylist)
    unit.M=temp

    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[{"role":"user",
          "content":"Please generate a short prompt based on the following(Less than 25 tokens)."+str(unit.M) + " " + str(unit.P)}],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )

    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp_2=''.join(mylist)
    unit.P=temp_2
    return unit


# Lamarckian Mutation
def working_out_task_prompt(unit: EvolutionUnit, **kwargs) -> EvolutionUnit:
    RANDOM_WORKING_OUT = random.sample(gsm8k_examples, 1)[0]
    completion = client.chat.completions.create(
      model="meta/llama3-8b-instruct",
      messages=[{"role":"user",
          "content":"Please generate a short prompt based on the following(Less than 25 tokens). I gave a friend an instruction and some advice. Here are the correct examples of his workings out "
          + RANDOM_WORKING_OUT['question'] + " " + RANDOM_WORKING_OUT['answer'] + " The instruction was: "
      }],
      temperature=0.5,
      top_p=1,
      max_tokens=526,
      stream=True
    )

    mylist=[]
    for chunk in completion:
      if chunk.choices[0].delta.content is not None:
        mylist.append(chunk.choices[0].delta.content)
    temp=''.join(mylist)
    unit.P=temp
    return unit

MUTATORS = [
    zero_order_prompt_gen,
    first_order_prompt_gen,
    lineage_based_mutation,
    zero_order_hypermutation,
    first_order_hypermutation,
    working_out_task_prompt
]

In [ ]:
def mutate(population=population):
  indices = [i for i in range(len(population.units))]
  random.shuffle(indices)
  pairs = [indices[2 * x:2 * x + 2] for x in range(len(indices) // 2)]

  # binary tourmanent genetic algorithm
  for i in range(len(pairs)):
    first_unit = population.units[pairs[i][0]]
    second_unit = population.units[pairs[i][1]]
    FIRST_WON = False
    if first_unit.fitness >= second_unit.fitness:
        # loser gets mutated.
        FIRST_WON = True
        mutation_input = second_unit
    else:
        mutation_input = first_unit

    data = {
        'unit': mutation_input,
        'elites': population.elites,
        'problem_description': population.problem_description,
    }

    # uniformly pick and call a random mutation operator on the losing unit
    random_mutator = random.sample(MUTATORS, 1)[0]
    print(f"MUTATING: {mutation_input} with {random_mutator.__name__}")

    random_mutator(**data)

# run 1

## init

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# 初始化当前时间目录以存储结果
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = os.path.join(os.getcwd(), "Output")
current_time_dir = os.path.join(output_dir, now)
if not os.path.exists(current_time_dir):
  os.makedirs(current_time_dir)
population.result_dir = current_time_dir

# 生成初始prompt
print("开始生成初始prompt")
prompts = []
for unit in population.units:
  template = f"{unit.T} {unit.M} INSTRUCTION: {population.problem_description} INSTRUCTION MUTANT = "
  messages = [
    {"role": "system",
    "content":"Please generate a short prompt based on the following(Less than 25 tokens)."},
    {"role": "user", "content": template},]
  outputs=pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,)
  prompts.append(outputs[0]["generated_text"][-1]['content'])


# 将生成的prompt存储到units.P中
assert len(prompts) == population.size, "size of google response to population is mismatched"
for i, item in enumerate(prompts):
    population.units[i].P = item
print(prompts)


开始生成初始prompt

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[
    '"Design a clever experiment to crack the code of this math puzzle! Think outside the box and come up with a 
creative way to measure the solution. What tools and variables will you use to solve the problem? Write a concise 
plan, including your hypothesis, methodology, and expected outcome. Finally, convert your answer to an Arabic 
numeral and get ready to reveal the solution!"',
    'Here\'s a creative modification of the instruction:\n\n"Embark on a mathematical treasure hunt! Brainstorm a 
treasure trove of ideas to tackle the problem, and then systematically dig into each solution, one by one, to 
uncover the hidden treasure - the correct answer. Write down each attempt as an Arabic numeral, and see which one 
leads you to the loot!"',
    '"Track your triumphs! To measure progress on this math word problem, try breaking it down into smaller, 
manageable chunks. As you solve each step, jot down your answers and reflect on how far you\'ve come. This will 
help you stay focused and motivated, allowing you to celebrate your small wins along the way. By the time you reach
the final solution, you\'ll be able to look back on your progress and see just how far you\'ve come!"',
    '"Streamline the Math Maze: Break down the problem into bite-sized chunks by identifying the key elements and 
relationships. Then, use simple arithmetic operations to unravel the mystery. Finally, present your solution as a 
crisp, concise Arabic numeral. Can you simplify the problem and reveal the answer?"',
    '"Embark on a thrilling adventure to crack the code of the mystical math realm! Conjure an experiment to 
unravel the enigma, and when you\'ve solved it, unleash your answer as a radiant Arabic numeral, shining bright 
like a beacon of mathematical mastery!"',
    '"Escape the Math Maze!"\n\nImagine you\'re trapped in a mystical realm where numbers reign supreme. Your only 
hope for escape lies in solving a mind-bending math problem. Grab your trusty calculator and get ready to outsmart 
the numerals!',
    '"Embark on a thrilling adventure to quantify the triumph! Imagine a magical meter that measures mastery, a 
progress pendulum that pivots with each puzzle solved. How would you calibrate this wondrous device to gauge your 
journey towards solving the math word problem?"',
    '"Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and
then Conquer it with the Answer in Arabic Numerals!"'
]

## processing

In [ ]:
result_list = handle_result(system_prompt=prompts, batch_example=question_set)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a clever math puzzle!

To design a clever experiment to crack the code, I'll propose a plan to solve this problem. Here it is:

**Hypothesis:** Carol will have a certain number of invitations left after sending 2 invitations.

**Methodology:**

1. Calculate the total number of invitations Carol has: 7 packs x 10 invitations per pack = 70 invitations
2. Subtract the number of invitations sent to 2 friends: 70 - 2 =?

**Expected Outcome:** We'll find the number of invitations left after sending 2 invitations.

**Solution:**

70 - 2 = 68

So, Carol will have 68 invitations left after sending invitations to 2 friends.

**Converting to an Arabic numeral:**

68.0 =>

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a delicious math puzzle! Let's design a clever experiment to crack the code.

**Hypothesis:**
We'll assume that the number of peaches Jake has is represented by the variable x.

**Methodology:**

1. We know Jill has 87 peaches. Let's use this as a reference point.
2. Steven has 18 more peaches than Jill, so we can write an equation: Steven's peaches = Jill's peaches + 18
3. Since Jake has 5 fewer peaches than Steven, we can write another equation: Jake's peaches = Steven's peaches - 5

**Equation:**
Substitute the first equation into the second one to get:

Jake's peaches = (Jill's peaches + 18) - 5

**Simplification:**
Substitute Jill's peaches with 87:

Jake's peaches = (87 + 18) - 5
Jake's peaches = 105 - 5
Jake's peaches = 100

**Question:** How many more peaches does Jake have than Jill?

**Answer:** Jake has 100 - 87 = 13 more peaches than Jill.

**Conversion to Arabic numeral:**

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's break it down:

The Razorback t-shirt shop made $51 from selling 3 t-shirts. We can set up an equation to represent this:

3x = 51

Where x is the cost of each t-shirt.

To solve for x, we can divide both sides of the equation by 3:

x = 51 ÷ 3
x = 17

So, the cost of each t-shirt is $17.

=> 17.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's simplify the problem:

Number of campers who went rowing in the morning = 62
Number of campers who went rowing in the afternoon = 39

To find the difference, subtract the number of campers who went rowing in the afternoon from the number of campers 
who went rowing in the morning:

62 - 39 = 23

So, 23 more campers went rowing in the morning than in the afternoon.

=> 23.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's break it down:

* Total number of students = 111
* Number of seats per bus = 3

To find the number of buses needed, we can divide the total number of students by the number of seats per bus:

111 ÷ 3 =?

Let's simplify the equation:

111 = 3x

Where x is the number of buses needed.

Solving for x:

x = 111 ÷ 3
x = 37

So, the answer is:

37 => 37.0

In [ ]:
evaluate_fitness(population=population, results=result_list, batch=question_set,num_evals=num_evals)

正在评价结果：0

正在评价结果：1

正在评价结果：2

正在评价结果：3

正在评价结果：4

...

正在评价结果：35

正在评价结果：36

正在评价结果：37

正在评价结果：38

正在评价结果：39

# run N

## simulation 1

In [ ]:
mutate(population=population)

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='How could I 
measure progress on this problem?' P='"Track your triumphs! To measure progress on this math word problem, try 
breaking it down into smaller, manageable chunks. As you solve each step, jot down your answers and reflect on how 
far you\'ve come. This will help you stay focused and motivated, allowing you to celebrate your small wins along 
the way. By the time you reach the final solution, you\'ll be able to look back on your progress and see just how 
far you\'ve come!"' fitness=0 history=[] with zero_order_prompt_gen

MUTATING: T='Just change this instruction to make it more fun, think WELL outside the box:' M='How could I measure 
progress on this problem?' P='"Embark on a thrilling adventure to quantify the triumph! Imagine a magical meter 
that measures mastery, a progress pendulum that pivots with each puzzle solved. How would you calibrate this 
wondrous device to gauge your journey towards solving the math word problem?"' fitness=0 history=[] with 
first_order_prompt_gen

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='How could I 
devise an experiment to help solve that problem?' P='"Design a clever experiment to crack the code of this math 
puzzle! Think outside the box and come up with a creative way to measure the solution. What tools and variables 
will you use to solve the problem? Write a concise plan, including your hypothesis, methodology, and expected 
outcome. Finally, convert your answer to an Arabic numeral and get ready to reveal the solution!"' fitness=0 
history=[] with zero_order_hypermutation

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='Make a list of
ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.' 
P='Here\'s a creative modification of the instruction:\n\n"Embark on a mathematical treasure hunt! Brainstorm a 
treasure trove of ideas to tackle the problem, and then systematically dig into each solution, one by one, to 
uncover the hidden treasure - the correct answer. Write down each attempt as an Arabic numeral, and see which one 
leads you to the loot!"' fitness=0 history=[] with working_out_task_prompt

### processing

In [ ]:
# 生成prompt的列表
prompts = []
for unit in population.units:
  prompts.append(unit.P)
print(prompts)

[
    '"Design a clever experiment to crack the code of this math puzzle! Think outside the box and come up with a 
creative way to measure the solution. What tools and variables will you use to solve the problem? Write a concise 
plan, including your hypothesis, methodology, and expected outcome. Finally, convert your answer to an Arabic 
numeral and get ready to reveal the solution!"',
    'Here is a short prompt based on your input:\n\nYour friend struggles with math, so you give them an 
instruction and some advice. They then use the correct examples to solve a problem about oranges being harvested. 
How do they figure out the total number of oranges harvested per day, given the number of sacks and the number of 
oranges per sack?',
    '',
    '"Streamline the Math Maze: Break down the problem into bite-sized chunks by identifying the key elements and 
relationships. Then, use simple arithmetic operations to unravel the mystery. Finally, present your solution as a 
crisp, concise Arabic numeral. Can you simplify the problem and reveal the answer?"',
    '"Embark on a thrilling adventure to crack the code of the mystical math realm! Conjure an experiment to 
unravel the enigma, and when you\'ve solved it, unleash your answer as a radiant Arabic numeral, shining bright 
like a beacon of mathematical mastery!"',
    '"Escape the Math Maze!"\n\nImagine you\'re trapped in a mystical realm where numbers reign supreme. Your only 
hope for escape lies in solving a mind-bending math problem. Grab your trusty calculator and get ready to outsmart 
the numerals!',
    'Here\'s a short prompt:\n\n"Design a progress meter to measure your journey in solving a math word problem. 
What indicators would you use to calibrate the meter, and how would you adjust it as you make progress?"',
    '"Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and
then Conquer it with the Answer in Arabic Numerals!"'
]

In [ ]:
result_list = handle_result(system_prompt=prompts, batch_example=question_set)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a clever math puzzle!

To crack the code, I'll design an experiment to solve the problem. Here's my plan:

**Hypothesis:** If Carol buys 7 packs of invitations with 10 invitations each, and sends invitations to only 2 
friends, the number of invitations left will be a function of the total number of invitations and the number of 
friends invited.

**Methodology:**

1. Calculate the total number of invitations Carol has: 7 packs x 10 invitations per pack = 70 invitations
2. Calculate the number of invitations sent to friends: 2 friends
3. Subtract the number of invitations sent from the total number of invitations to find the number of invitations 
left: 70 - 2 =?

**Expected Outcome:** I expect the number of invitations left to be a decimal value, as it's likely that Carol 
won't send an exact number of invitations that divides evenly into the total.

**Solution:**

Let x be the number of invitations left. Then, the equation is:

70 - 2 = x

Solving for x, I get:

70 - 2 => 68

Since Carol sent invitations to only 2 friends, the number of invitations left is:

68.0

So, the answer is

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a delightful math puzzle! Let's design an experiment to crack the code.

**Hypothesis:** We will find the number of peaches Jake has by using the given information and then subtract it 
from Jill's peaches to find the difference.

**Methodology:**

1. Let x be the number of peaches Jake has.
2. Since Jake has 5 fewer peaches than Steven, we can write an equation: x = Steven's peaches - 5
3. Steven has 18 more peaches than Jill, so we can write another equation: Steven's peaches = Jill's peaches + 18
4. Since Jill has 87 peaches, we can substitute this value into the second equation: Steven's peaches = 87 + 18 = 
105
5. Now, substitute Steven's peaches into the first equation: x = 105 - 5 = 100
6. Finally, subtract Jill's peaches from Jake's peaches to find the difference: Jake's peaches - Jill's peaches = 
100 - 87 = 13

**Expected Outcome:** Jake has 13 more peaches than Jill.

**Solution:** 13.0 =>

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's break it down:

The Razorback t-shirt shop made $51 from selling 3 t-shirts. We can set up an equation to represent this:

3x = 51

Where x is the cost of each t-shirt.

To solve for x, we can divide both sides of the equation by 3:

x = 51 ÷ 3
x = 17

So, the cost of each t-shirt is $17.00.

=> 17.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem!

Let's simplify the problem by finding the difference between the number of campers who went rowing in the morning 
and the number who went rowing in the afternoon.

Morning: 62 campers
Afternoon: 39 campers

Subtract the number of campers in the afternoon from the number of campers in the morning:

62 - 39 = 23

So, 23 more campers went rowing in the morning than in the afternoon.

Transform the answer into a solvable puzzle by converting it into an equation:

x = 23

Conquer the puzzle with the answer in Arabic numerals:

23 => 23.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's break it down:

* There are 72 classrooms, but we need to find the number of buses needed to take the students, not the classrooms.
* Each bus has 3 seats, so we need to find the number of buses needed to accommodate all 111 students.
* Let's use the variable x to represent the number of buses needed.

We can set up an equation:

3x ≥ 111 (since each bus has 3 seats, and we need to accommodate all 111 students)

To solve for x, we can divide both sides by 3:

x ≥ 111 ÷ 3
x ≥ 37

Since we can't have a fraction of a bus, we'll round up to the nearest whole number:

x = 38

So, the answer is:

38 => 37.3

In [ ]:
evaluate_fitness(population=population, results=result_list, batch=question_set, num_evals=num_evals)

正在评价结果：0

正在评价结果：1

正在评价结果：2

正在评价结果：3

正在评价结果：4

...

正在评价结果：35

正在评价结果：36

正在评价结果：37

正在评价结果：38

正在评价结果：39

## simulation 2

In [ ]:
mutate(population=population)

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='How could I 
measure progress on this problem?' P='' fitness=0 history=[] with working_out_task_prompt

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='Make a list of
ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.' 
P='Here is a short prompt based on your input:\n\nYour friend struggles with math, so you give them an instruction 
and some advice. They then use the correct examples to solve a problem about oranges being harvested. How do they 
figure out the total number of oranges harvested per day, given the number of sacks and the number of oranges per 
sack?' fitness=0 history=[] with working_out_task_prompt

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='How can I 
simplify the problem so that it is easier to solve?' P='"Streamline the Math Maze: Break down the problem into 
bite-sized chunks by identifying the key elements and relationships. Then, use simple arithmetic operations to 
unravel the mystery. Finally, present your solution as a crisp, concise Arabic numeral. Can you simplify the 
problem and reveal the answer?"' fitness=0 history=[] with lineage_based_mutation

MUTATING: T='Just change this instruction to make it more fun, think WELL outside the box:' M='How could I devise 
an experiment to help solve that problem?' P='"Embark on a thrilling adventure to crack the code of the mystical 
math realm! Conjure an experiment to unravel the enigma, and when you\'ve solved it, unleash your answer as a 
radiant Arabic numeral, shining bright like a beacon of mathematical mastery!"' fitness=0 history=[] with 
zero_order_prompt_gen

### processing

In [ ]:
# 生成prompt的列表
prompts = []
for unit in population.units:
  prompts.append(unit.P)
print(prompts)

[
    '"Design a clever experiment to crack the code of this math puzzle! Think outside the box and come up with a 
creative way to measure the solution. What tools and variables will you use to solve the problem? Write a concise 
plan, including your hypothesis, methodology, and expected outcome. Finally, convert your answer to an Arabic 
numeral and get ready to reveal the solution!"',
    "Here is a short prompt based on your input:\n\nYour friend asked for help with a math problem. You provided an
instruction and some advice, and now they've completed the correct solution. What was the instruction you gave, and
how did your friend arrive at the correct answer?",
    'Here is a short prompt based on your input:\n\n"Your friend struggled with a math problem and asked for your 
help. You gave them an instruction and some advice, and now they\'ve shown you their correct workings. The problem 
was: In a school, there are 308 girls and 318 boys. There are also 36 teachers. How many pupils are there in that 
school? What was the instruction you gave your friend, and how did they solve the problem?"',
    'Here is a short prompt based on the provided text:\n\n**Unlock the Secrets of the Math Maze:**\n\nTransform 
the math problem into a solvable puzzle by simplifying the issue. Conquer the challenge by finding the answer in 
Arabic numerals.',
    '',
    '"Escape the Math Maze!"\n\nImagine you\'re trapped in a mystical realm where numbers reign supreme. Your only 
hope for escape lies in solving a mind-bending math problem. Grab your trusty calculator and get ready to outsmart 
the numerals!',
    'Here\'s a short prompt:\n\n"Design a progress meter to measure your journey in solving a math word problem. 
What indicators would you use to calibrate the meter, and how would you adjust it as you make progress?"',
    '"Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and
then Conquer it with the Answer in Arabic Numerals!"'
]

In [ ]:
result_list = handle_result(system_prompt=prompts, batch_example=question_set)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a clever math puzzle! Let's design an experiment to crack the code.

**Hypothesis:**
Carol will have some invitations left over after sending out invitations to only 2 friends.

**Methodology:**

1. Calculate the total number of invitations Carol has: 7 packs x 10 invitations/pack = 70 invitations
2. Calculate the number of invitations sent to 2 friends: 2 friends x? invitations/friend (we'll come back to this 
later)
3. Subtract the number of invitations sent from the total number of invitations to find the number of invitations 
left over.

**Variables:**

* x: the number of invitations sent to each friend
* Total invitations: 70
* Invitations sent: 2x
* Invitations left over:?

**Equation:**
70 - 2x =?

**Expected Outcome:**
We expect to find the number of invitations left over by solving the equation.

**Solution:**
Let's solve the equation:

70 - 2x =?

We want to find the value of x, so we'll add 2x to both sides:

70 = 2x + x

Combine like terms:

70 = 3x

Now, divide both sides by 3

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a delightful math puzzle! Let's design a clever experiment to crack the code.

Hypothesis:
We can solve the problem by finding the number of peaches Steven has, and then subtracting Jake's peaches from 
Jill's peaches.

Methodology:

1. Start by using the information that Jill has 87 peaches.
2. Since Steven has 18 more peaches than Jill, we can write an equation: Steven's peaches = Jill's peaches + 18
3. Substitute Jill's peaches (87) into the equation: Steven's peaches = 87 + 18
4. Calculate Steven's peaches: Steven's peaches = 105
5. Since Jake has 5 fewer peaches than Steven, we can write an equation: Jake's peaches = Steven's peaches - 5
6. Substitute Steven's peaches (105) into the equation: Jake's peaches = 105 - 5
7. Calculate Jake's peaches: Jake's peaches = 100
8. Finally, find the difference between Jake's peaches and Jill's peaches to get the answer: Jake's peaches - 
Jill's peaches = 100 - 87

Expected Outcome:
The

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem!

Let's break it down:

* The Razorback t-shirt shop made $51 from selling 3 t-shirts.
* We need to find the cost of each t-shirt.

Let's create an equation:

Total Cost = Number of T-shirts × Cost per T-shirt

We know the total cost is $51, and the number of t-shirts is 3. So, we can write:

51 = 3 × Cost per T-shirt

To simplify the equation, we can divide both sides by 3:

51 ÷ 3 = Cost per T-shirt

Cost per T-shirt = 17

So, the cost of each t-shirt is $17.

=> 17.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's simplify the problem:

Number of campers in the morning: 62
Number of campers in the afternoon: 39

To find the difference, subtract the number of campers in the afternoon from the number of campers in the morning:

62 - 39 =?

=> 23.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's simplify the problem: The school has 111 students and 3 seats per bus. To find the number of buses needed, we
can divide the total number of students by the number of seats per bus.

111 students ÷ 3 seats/bus =?

=> 37.0

In [ ]:
evaluate_fitness(population=population, results=result_list, batch=question_set, num_evals=num_evals)

正在评价结果：0

正在评价结果：1

正在评价结果：2

正在评价结果：3

正在评价结果：4

...

正在评价结果：35

正在评价结果：36

正在评价结果：37

正在评价结果：38

正在评价结果：39

## simulation 3

In [ ]:
mutate(population=population)

MUTATING: T='Just change this instruction to make it more fun, think WELL outside the box:' M='How could I devise 
an experiment to help solve that problem?' P='' fitness=0 history=[] with zero_order_prompt_gen

MUTATING: T='Just change this instruction to make it more fun, think WELL outside the box:' M='Make a list of ideas
for solving this problem, and apply them one by one to the problem to see if any progress can be made.' P='"Escape 
the Math Maze!"\n\nImagine you\'re trapped in a mystical realm where numbers reign supreme. Your only hope for 
escape lies in solving a mind-bending math problem. Grab your trusty calculator and get ready to outsmart the 
numerals!' fitness=0 history=[] with first_order_prompt_gen

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='How could I 
measure progress on this problem?' P='Here is a short prompt based on your input:\n\n"Your friend struggled with a 
math problem and asked for your help. You gave them an instruction and some advice, and now they\'ve shown you 
their correct workings. The problem was: In a school, there are 308 girls and 318 boys. There are also 36 teachers.
How many pupils are there in that school? What was the instruction you gave your friend, and how did they solve the
problem?"' fitness=0 history=[] with lineage_based_mutation

MUTATING: T='Modify the following instruction creatively, giving some advice on how to solve it:' M='Here is a 
short prompt:\n\nA company is considering two different strategies to launch a new product. Strategy A involves 
investing $100,000 in a large-scale marketing campaign, while Strategy B involves investing $50,000 in social media
advertising. However, Strategy A has a higher risk of failure (20% chance of success) compared to Strategy B (40% 
chance of success). If the product is successful, it will generate $500,000 in revenue. What is the expected return
on investment for each strategy, and which one should the company choose?' P='"Design a clever experiment to crack 
the code of this math puzzle! Think outside the box and come up with a creative way to measure the solution. What 
tools and variables will you use to solve the problem? Write a concise plan, including your hypothesis, 
methodology, and expected outcome. Finally, convert your answer to an Arabic numeral and get ready to reveal the 
solution!"' fitness=0 history=[] with zero_order_hypermutation

In [ ]:
# 生成prompt的列表
prompts = []
for unit in population.units:
  prompts.append(unit.P)

result_list = handle_result(system_prompt=prompts, batch_example=question_set)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a clever math puzzle! Let's design an experiment to crack the code.

**Hypothesis:** Carol will have some invitations left over after sending out invitations to only 2 friends.

**Methodology:**

1. Calculate the total number of invitations Carol has: 7 packs x 10 invitations/pack = 70 invitations
2. Determine the number of invitations Carol sends to 2 friends: 2 friends x? invitations/friend (we'll come back 
to this)
3. Calculate the number of invitations left over: 70 invitations -? invitations sent =? invitations left

**Variables:**

* x: the number of invitations Carol sends to each friend
* 70: the total number of invitations Carol has
* 2: the number of friends Carol sends invitations to

**Expected Outcome:** We expect to find the number of invitations left over after Carol sends out invitations to 2 
friends.

**Equation:**

70 - 2x =? invitations left

**Solution:** To find the number of invitations left, we need to find the value of x. Let's assume Carol sends x 
invitations to each friend. Then, the total number of invitations sent is 2x. Subtracting this from the total 
number of invitations, we

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a delicious math puzzle!

Let's design a clever experiment to crack the code:

**Hypothesis:** We will find the number of peaches Jake has and then subtract Jill's peaches to find the 
difference.

**Methodology:**

1. Let x be the number of peaches Jake has.
2. Since Jake has 5 fewer peaches than Steven, Steven has x + 5 peaches.
3. Since Steven has 18 more peaches than Jill, Jill has (x + 5) - 18 = x - 13 peaches.
4. We know Jill has 87 peaches, so we can set up the equation: x - 13 = 87.
5. To solve for x, we add 13 to both sides: x = 87 + 13 = 100.
6. Now that we have x, we can find the number of peaches Jake has: x = 100.
7. To find the difference between Jake's peaches and Jill's peaches, we subtract: 100 - 87 = 13.

**Expected Outcome:** Jake has 13 more peaches than Jill.

**Solution:** 13.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem!

Let's break it down:

* The Razorback t-shirt shop made $51 from selling 3 t-shirts.
* Let's find the cost of each t-shirt.

We can set up an equation:

Cost per t-shirt × 3 = 51

Since we don't know the cost per t-shirt yet, let's call it x.

3x = 51

Now, let's simplify the equation by dividing both sides by 3:

x = 51 ÷ 3

x = 17

So, the cost of each t-shirt is $17.

=> 17.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem!

Let's simplify the problem by finding the difference between the number of campers who went rowing in the morning 
and the number of campers who went rowing in the afternoon.

Morning campers - 62
Afternoon campers - 39

To find the difference, subtract the afternoon campers from the morning campers:

62 - 39 = 23

So, 23 more campers went rowing in the morning than in the afternoon.

Conquer it with the Answer in Arabic Numerals!

23 => 23.0

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Embark on a Quest for Clarity: Transform the Math Maze into a Solvable Puzzle by Simplifying the Problem, and then 
Conquer it with the Answer in Arabic Numerals!

Let's break it down:

* The school has 72 classrooms, but that's not relevant to the number of buses needed.
* There are 111 students in the school.
* Each school bus can hold 3 students.

To find the number of buses needed, we need to divide the total number of students by the number of students each 
bus can hold:

111 students ÷ 3 students per bus =?

Simplifying the equation:

37.0 => 37.0

In [ ]:
evaluate_fitness(population=population, results=result_list, batch=question_set, num_evals=num_evals)

正在评价结果：0

正在评价结果：1

正在评价结果：2

正在评价结果：3

正在评价结果：4

...

正在评价结果：35

正在评价结果：36

正在评价结果：37

正在评价结果：38

正在评价结果：39